# 1.RFM 개념

**: 고객 세분화 방법**

- Recency: 마지막 구매일이 언제 인가
- Frequency: 얼마나 자주 구매했는가
- Monetary: 구매 금액이 얼마나 되는가

- R,F,M 각 값에 가중치를 부여하고 나누는 기준을 정한다.
- 제품 특성에 따라 달라진다.

**R/F 크로스 비교 분석**

- Recency, Frequency 평점이 모두 높은 경우
    - 최근 구매한 이력이 많고 빈번하게 구매를 하고 있기 때문에 단골고객으로 분류
    
- Recency 평점은 높으나 Frequency 평점은 낮은 경우
    - 최근에 구매했으나 그 이전 구매 이력이 없기 때문에 신규 고객으로 분류
    
- Frequency 평점은 높으나 Recency 평점이 낮은 경우
    - 과거엔 구매를 빈번하게 했으나 최근 구매 이력이 없기 때문에 어떠한 불신이 생긴 경우이기 때문에 불신 발생고객으로 분류

# 2.데이터 로드

In [18]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

session = pd.read_csv("data/Session.csv")
product = pd.read_csv("data/Product.csv")
master = pd.read_csv("data/Master.csv")

data = pd.merge(product, session, how = 'left', on = ['CLNT_ID','SESS_ID'])
data = pd.merge(data, master, how = 'left', on = ['PD_C'])

print(len(data))
data.sample(3)

5024906


,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
3487709,4841536,3284315,16,517206,색상:PO-A7-22-그레이/사이즈:XL36,[파파브로],"18,800",1,30,20180806,14.0,403,desktop,Daejeon,Daejeon,남자 정장 캐주얼 스판 슬랙스 9부바지 PO-A7-22-그레이 - PO-A7-22-...,남성의류,남성의류하의,남성캐주얼바지
1209936,3223306,5727197,46,459040,색상/사이즈:핑크/240 / 1개,이랜드주니어,"7,900",1,369,20180626,48.0,460,mobile,Gyeongsangnam-do,Gimhae-si,{16H2_18S0236}데비탐스 - 핑크/230,패션잡화,유아동화,유아동슬립온
3186955,2176128,2930854,122,1838,1개,패트론(시계),"212,000",1,129,20180812,235.0,"3,025",mobile,Gyeonggi-do,Goyang-si,패트론파리(PATRON PARIS) 여성메탈시계 (PR201201F_CB),패션잡화,시계,여성시계
11733,4367987,4450527,24,664366,2개,헬스익스텐션,900,2,7,20180717,41.0,"1,629",tablet,Gyeongsangnam-do,Tongyeong-si,◆[빠른배송]그레인프리 독 치킨 앤 칠면조 450g / 유통기한 임박상품,원예/애완,애견용품,애견사료
99531,4284558,940670,66,813023,2개,에뛰드하우스,"1,050",2,33,20180915,72.0,504,mobile,Gyeonggi-do,Yongin-si,네일 리무버 일반손톱용 (마일드) 100ml,화장품/뷰티케어,메이크업,일반네일/케어류


컬럼 선택

In [19]:
data =  data[['CLNT_ID','SESS_ID','SESS_DT','PD_C','PD_BUY_AM','CLAC1_NM','CLAC2_NM','CLAC3_NM']]
data.sample(3)

,CLNT_ID,SESS_ID,SESS_DT,PD_C,PD_BUY_AM,CLAC1_NM,CLAC2_NM,CLAC3_NM
897104,3730171,2946985,20180812,443887,"49,000",여성의류,여성의류상의,여성티셔츠/탑
636268,2202475,3252179,20180806,9617,"47,000",원예/애완,고양이용품,고양이사료
4711476,1752184,7124818,20180603,607240,"34,900",화장품/뷰티케어,향수,여성향수


제품군별 특성이 다르기 때문에 특정 카테고리를 정한다.

In [41]:
women_clothing = data[data['CLAC1_NM'] == '여성의류']

# 3.데이터 정제

최근 구매일, 구매빈도, 구매금액 컬럼 만들기

In [42]:
# 컬럼 선택
women_clothing = women_clothing[['CLNT_ID','SESS_DT','PD_BUY_AM','CLAC1_NM']]
# 날짜 변환
women_clothing['SESS_DT'] = pd.to_datetime(women_clothing['SESS_DT'], format='%Y%m%d')

women_clothing.head()

,CLNT_ID,SESS_DT,PD_BUY_AM,CLAC1_NM
12,4154576,2018-07-20,"39,000",여성의류
17,4167192,2018-04-03,"39,000",여성의류
19,4170352,2018-07-14,"39,000",여성의류
25,4180381,2018-04-16,"39,000",여성의류
29,4185920,2018-06-11,"39,000",여성의류


In [44]:
# 최근 구매일
# women_clothing.groupby('CLNT_ID')['SESS_DT'].max()

# 구매 빈도
women_clothing.groupby('CLNT_ID').size()

CLNT_ID
71          1
149         2
306         2
379         2
441         2
451        16
460         3
461         1
463         3
498         1
502         1
520         1
556        37
627         2
678         2
684         5
714         6
741         2
745         3
813         8
864        13
871         2
873         2
905         5
918        13
958         4
970         1
984         3
988         3
991         5
           ..
6591661     7
6591674     2
6591732     1
6591741     1
6591746     1
6591771     1
6591806     6
6591868     3
6591871     5
6591874     1
6591882     1
6591903     5
6591907     2
6591934     1
6592089     1
6592103     5
6592197     1
6592213     2
6592231     1
6592259     1
6592291     4
6592322     1
6592335     1
6592389     2
6592402     7
6592405     2
6592461     1
6592475    11
6592537     2
6592553     2
Length: 193022, dtype: int64

In [45]:
6592475

women_clothing[women_clothing['CLNT_ID']==6592475]

,CLNT_ID,SESS_DT,PD_BUY_AM,CLAC1_NM
77666,6592475,2018-09-17,"39,000",여성의류
354761,6592475,2018-09-05,"44,100",여성의류
542427,6592475,2018-09-05,"44,100",여성의류
1001373,6592475,2018-09-12,"49,000",여성의류
1164093,6592475,2018-09-05,"54,000",여성의류
1164094,6592475,2018-09-05,"54,000",여성의류
1944235,6592475,2018-09-03,"72,500",여성의류
2022664,6592475,2018-09-04,"10,000",여성의류
3573558,6592475,2018-09-03,"19,000",여성의류
3600203,6592475,2018-09-04,"19,000",여성의류
